In [1]:
import pandas as pd
import argparse
import logging
import json
import os
import ast
import pathlib
import fiona
import numpy as np
import geopandas as gpd
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from geopandas.tools import geocode
from shapely.geometry import Point
import matplotlib.pyplot as plt
import requests
from zipfile import ZipFile
from tqdm import trange, tqdm
from operator import itemgetter
import seaborn as sns

In [2]:
pathlib.Path()
NOTEBOOK_PATH = pathlib.Path().resolve()
p = NOTEBOOK_PATH.parent
DATA_DIRECTORY = p / "data"
sup_file = DATA_DIRECTORY / "support_data" 

In [3]:
STEM_timeseries = pd.read_excel(sup_file/"STEM_timeseries.xlsx", header=0, sheet_name ="STEM_monthly")
STEM_timeseries

,LABEL,Load_type,Sector,dayofweek,Hour,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,air-conditioning-demand-profile,AC,Services,WD,0,0.000000,0.000000,0.002010,0.002010,0.002010,0.004040,0.004040,0.004040,0.002010,0.002010,0.002010,0.000000
1,air-conditioning-demand-profile,AC,Services,WD,1,0.000000,0.000000,0.002078,0.002078,0.002078,0.004217,0.004217,0.004217,0.002078,0.002078,0.002078,0.000000
2,air-conditioning-demand-profile,AC,Services,WD,2,0.000000,0.000000,0.002150,0.002150,0.002150,0.004317,0.004317,0.004317,0.002150,0.002150,0.002150,0.000000
3,air-conditioning-demand-profile,AC,Services,WD,3,0.000000,0.000000,0.002460,0.002460,0.002460,0.004753,0.004753,0.004753,0.002460,0.002460,0.002460,0.000000
4,air-conditioning-demand-profile,AC,Services,WD,4,0.000000,0.000000,0.002902,0.002902,0.002902,0.005367,0.005367,0.005367,0.002902,0.002902,0.002902,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,lighting-demand-profile,ELEC_LIGHT,Residential,WE,19,0.010327,0.010327,0.009888,0.009888,0.009888,0.001538,0.001538,0.001538,0.009448,0.009448,0.009448,0.010327
668,lighting-demand-profile,ELEC_LIGHT,Residential,WE,20,0.009888,0.009888,0.009448,0.009448,0.009448,0.001538,0.001538,0.001538,0.009448,0.009448,0.009448,0.009888
669,lighting-demand-profile,ELEC_LIGHT,Residential,WE,21,0.008789,0.008789,0.008789,0.008789,0.008789,0.005274,0.005274,0.005274,0.008789,0.008789,0.008789,0.008789
670,lighting-demand-profile,ELEC_LIGHT,Residential,WE,22,0.007251,0.007251,0.007251,0.007251,0.007251,0.006372,0.006372,0.006372,0.007251,0.007251,0.007251,0.007251


In [4]:
# Create a DateTime index with hourly frequency for the year 2020
start_date = "2020-01-01 00:00"
end_date = "2020-12-31 23:00"
date_range = pd.date_range(start=start_date, end=end_date, freq="H")

# Format the DateTime objects as strings with the desired format
formatted_dates = date_range.strftime("%d/%m/%Y %H:%M")

# Create a DataFrame with the formatted dates
TI_timeseries = pd.DataFrame({"datetime": date_range, "formatted_date": formatted_dates})

In [5]:
#TI_timeseries["dt"] = pd.to_datetime(TI_timeseries["datetime"])
TI_timeseries["dayofweek"] = TI_timeseries["datetime"].dt.strftime("%w")
TI_timeseries["month"] = TI_timeseries["datetime"].dt.strftime("%b")
TI_timeseries["hour"] = TI_timeseries["datetime"].dt.hour
TI_timeseries[["res_sh","res_dhw","res_elec_app","res_elec_light","ind_sh","ind_dhw","ind_hth", "ind_elec","ser_sh_dhw","ser_elec","agr_elec"]] = np.nan

In [6]:
# Residential
STEM_res_SH_WD = STEM_timeseries[(STEM_timeseries["Sector"] == "Residential - Existing single family houses") & (STEM_timeseries["Load_type"] == "SH") & (STEM_timeseries["dayofweek"] == "WD")]
STEM_res_SH_WE = STEM_timeseries[(STEM_timeseries["Sector"] == "Residential - Existing single family houses") & (STEM_timeseries["Load_type"] == "SH") & (STEM_timeseries["dayofweek"] == "WE")]
STEM_res_DHW_WD = STEM_timeseries[(STEM_timeseries["Sector"] == "Residential") & (STEM_timeseries["Load_type"] == "DHW") & (STEM_timeseries["dayofweek"] == "WD")]
STEM_res_DHW_WE = STEM_timeseries[(STEM_timeseries["Sector"] == "Residential") & (STEM_timeseries["Load_type"] == "DHW") & (STEM_timeseries["dayofweek"] == "WE")]
STEM_res_ELEC_APP_WD = STEM_timeseries[(STEM_timeseries["Sector"] == "Residential") & (STEM_timeseries["Load_type"] == "ELEC_APP") & (STEM_timeseries["dayofweek"] == "WD")]
STEM_res_ELEC_APP_WE = STEM_timeseries[(STEM_timeseries["Sector"] == "Residential") & (STEM_timeseries["Load_type"] == "ELEC_APP") & (STEM_timeseries["dayofweek"] == "WE")]
STEM_res_ELEC_LIGHT_WD = STEM_timeseries[(STEM_timeseries["Sector"] == "Residential") & (STEM_timeseries["Load_type"] == "ELEC_LIGHT") & (STEM_timeseries["dayofweek"] == "WD")]
STEM_res_ELEC_LIGHT_WE = STEM_timeseries[(STEM_timeseries["Sector"] == "Residential") & (STEM_timeseries["Load_type"] == "ELEC_LIGHT") & (STEM_timeseries["dayofweek"] == "WE")]  #NEED TO CORRECT

# Agriculture
# X_agr_ELEC_WD = STEM_timeseries[(STEM_timeseries["Sector"] == "Agriculture") & (STEM_timeseries["Load_type"] == "ELEC") & (STEM_timeseries["dayofweek"] == "WD")] #NEED TO FIND REF
# X_agr_ELEC_WE = STEM_timeseries[(STEM_timeseries["Sector"] == "Agriculture") & (STEM_timeseries["Load_type"] == "ELEC") & (STEM_timeseries["dayofweek"] == "WE")] #NEED TO FIND REF

# Industry
STEM_ind_SH_WD = STEM_timeseries[(STEM_timeseries["Sector"] == "Industry") & (STEM_timeseries["Load_type"] == "SH") & (STEM_timeseries["dayofweek"] == "WD")]
STEM_ind_SH_WE = STEM_timeseries[(STEM_timeseries["Sector"] == "Industry") & (STEM_timeseries["Load_type"] == "SH") & (STEM_timeseries["dayofweek"] == "WE")]
STEM_ind_DHW_WD = STEM_timeseries[(STEM_timeseries["Sector"] == "Industry") & (STEM_timeseries["Load_type"] == "DHW") & (STEM_timeseries["dayofweek"] == "WD")]
STEM_ind_DHW_WE = STEM_timeseries[(STEM_timeseries["Sector"] == "Industry") & (STEM_timeseries["Load_type"] == "DHW") & (STEM_timeseries["dayofweek"] == "WE")]
STEM_ind_HTH_WD = STEM_timeseries[(STEM_timeseries["Sector"] == "Industry") & (STEM_timeseries["Load_type"] == "HTH") & (STEM_timeseries["dayofweek"] == "WD")]
STEM_ind_HTH_WE = STEM_timeseries[(STEM_timeseries["Sector"] == "Industry") & (STEM_timeseries["Load_type"] == "HTH") & (STEM_timeseries["dayofweek"] == "WE")]
# X_ind_ELEC_WD = STEM_timeseries[(STEM_timeseries["Sector"] == "Industry") & (STEM_timeseries["Load_type"] == "ELEC") & (STEM_timeseries["dayofweek"] == "WD")] #NEED TO FIND REF
# X_ind_ELEC_WE = STEM_timeseries[(STEM_timeseries["Sector"] == "Industry") & (STEM_timeseries["Load_type"] == "ELEC") & (STEM_timeseries["dayofweek"] == "WE")] #NEED TO FIND REF

# Services
STEM_ser_SH_DHW_WD = STEM_timeseries[(STEM_timeseries["Sector"] == "Services") & (STEM_timeseries["Load_type"] == "SH") & (STEM_timeseries["dayofweek"] == "WD")]
STEM_ser_SH_DHW_WE = STEM_timeseries[(STEM_timeseries["Sector"] == "Services") & (STEM_timeseries["Load_type"] == "SH") & (STEM_timeseries["dayofweek"] == "WE")]
# X_ser_ELEC_WD = STEM_timeseries[(STEM_timeseries["Sector"] == "Services") & (STEM_timeseries["Load_type"] == "ELEC") & (STEM_timeseries["dayofweek"] == "WD")] #NEED TO FIND REF 
# X_ser_ELEC_WE = STEM_timeseries[(STEM_timeseries["Sector"] == "Services") & (STEM_timeseries["Load_type"] == "ELEC") & (STEM_timeseries["dayofweek"] == "WE")] #NEED TO FIND REF


In [7]:
## Make normalised timeseries based on STEM values - works!
for i in range(len(TI_timeseries)):
    if TI_timeseries.at[i,"dayofweek"]== "0" or TI_timeseries.at[i,"dayofweek"]== "6":
        TI_timeseries.at[i,"res_sh"] = STEM_res_SH_WE.loc[STEM_res_SH_WE["Hour"] == TI_timeseries.at[i,"hour"],TI_timeseries.at[i,"month"]]
        TI_timeseries.at[i,"res_dhw"] = STEM_res_DHW_WE.loc[STEM_res_DHW_WE["Hour"] == TI_timeseries.at[i,"hour"],TI_timeseries.at[i,"month"]]
        TI_timeseries.at[i,"res_elec_app"] = STEM_res_ELEC_APP_WE.loc[STEM_res_ELEC_APP_WE["Hour"] == TI_timeseries.at[i,"hour"],TI_timeseries.at[i,"month"]]
        TI_timeseries.at[i,"res_elec_light"] = STEM_res_ELEC_LIGHT_WE.loc[STEM_res_ELEC_LIGHT_WE["Hour"] == TI_timeseries.at[i,"hour"],TI_timeseries.at[i,"month"]]
        
        TI_timeseries.at[i,"ind_sh"] = STEM_ind_SH_WE.loc[STEM_ind_SH_WE["Hour"] == TI_timeseries.at[i,"hour"],TI_timeseries.at[i,"month"]]
        TI_timeseries.at[i,"ind_dhw"] = STEM_ind_DHW_WE.loc[STEM_ind_DHW_WE["Hour"] == TI_timeseries.at[i,"hour"],TI_timeseries.at[i,"month"]]
        TI_timeseries.at[i,"ind_hth"] = STEM_ind_HTH_WE.loc[STEM_ind_HTH_WE["Hour"] == TI_timeseries.at[i,"hour"],TI_timeseries.at[i,"month"]]
#        TI_timeseries.at[i,"ind_elec"] = STEM_ind_ELEC_WE.loc[STEM_ind_ELEC_WE["Hour"] == TI_timeseries.at[i,"hour"],TI_timeseries.at[i,"month"]]
        
        TI_timeseries.at[i,"ser_sh_dhw"] = STEM_ser_SH_DHW_WE.loc[STEM_ser_SH_DHW_WE["Hour"] == TI_timeseries.at[i,"hour"],TI_timeseries.at[i,"month"]]
#        TI_timeseries.at[i,"ser_elec"] = STEM_ser_ELEC_WE.loc[STEM_ser_ELEC_WE["Hour"] == TI_timeseries.at[i,"hour"],TI_timeseries.at[i,"month"]]

#        TI_timeseries.at[i,"agr_elec"] = STEM_agr_ELEC_WE.loc[STEM_agr_ELEC_WE["Hour"] == TI_timeseries.at[i,"hour"],TI_timeseries.at[i,"month"]]

    else:
        TI_timeseries.at[i,"res_sh"] = STEM_res_SH_WD.loc[STEM_res_SH_WD["Hour"] == TI_timeseries.at[i,"hour"],TI_timeseries.at[i,"month"]]
        TI_timeseries.at[i,"res_dhw"] = STEM_res_DHW_WD.loc[STEM_res_DHW_WD["Hour"] == TI_timeseries.at[i,"hour"],TI_timeseries.at[i,"month"]]
        TI_timeseries.at[i,"res_elec_app"] = STEM_res_ELEC_APP_WD.loc[STEM_res_ELEC_APP_WD["Hour"] == TI_timeseries.at[i,"hour"],TI_timeseries.at[i,"month"]]
        TI_timeseries.at[i,"res_elec_light"] = STEM_res_ELEC_LIGHT_WD.loc[STEM_res_ELEC_LIGHT_WD["Hour"] == TI_timeseries.at[i,"hour"],TI_timeseries.at[i,"month"]]
        
        TI_timeseries.at[i,"ind_sh"] = STEM_ind_SH_WD.loc[STEM_ind_SH_WD["Hour"] == TI_timeseries.at[i,"hour"],TI_timeseries.at[i,"month"]]
        TI_timeseries.at[i,"ind_dhw"] = STEM_ind_DHW_WD.loc[STEM_ind_DHW_WD["Hour"] == TI_timeseries.at[i,"hour"],TI_timeseries.at[i,"month"]]
        TI_timeseries.at[i,"ind_hth"] = STEM_ind_HTH_WD.loc[STEM_ind_HTH_WD["Hour"] == TI_timeseries.at[i,"hour"],TI_timeseries.at[i,"month"]]
#        TI_timeseries.at[i,"ind_elec"] = STEM_ind_ELEC_WD.loc[STEM_ind_ELEC_WD["Hour"] == TI_timeseries.at[i,"hour"],TI_timeseries.at[i,"month"]]
        
        TI_timeseries.at[i,"ser_sh_dhw"] = STEM_ser_SH_DHW_WD.loc[STEM_ser_SH_DHW_WD["Hour"] == TI_timeseries.at[i,"hour"],TI_timeseries.at[i,"month"]]
#        TI_timeseries.at[i,"ser_elec"] = STEM_ser_ELEC_WD.loc[STEM_ser_ELEC_WD["Hour"] == TI_timeseries.at[i,"hour"],TI_timeseries.at[i,"month"]]

#        TI_timeseries.at[i,"agr_elec"] = STEM_agr_ELEC_WD.loc[STEM_agr_ELEC_WD["Hour"] == TI_timeseries.at[i,"hour"],TI_timeseries.at[i,"month"]]

C:\Users\jalomi.maayantardif\AppData\Local\Temp\ipykernel_9552\955576227.py:20: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  TI_timeseries.at[i,"res_sh"] = STEM_res_SH_WD.loc[STEM_res_SH_WD["Hour"] == TI_timeseries.at[i,"hour"],TI_timeseries.at[i,"month"]]
C:\Users\jalomi.maayantardif\AppData\Local\Temp\ipykernel_9552\955576227.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  TI_timeseries.at[i,"res_dhw"] = STEM_res_DHW_WD.loc[STEM_res_DHW_WD["Hour"] == TI_timeseries.at[i,"hour"],TI_timeseries.at[i,"month"]]
C:\Users\jalomi.maayantardif\AppData\Local\Temp\ipykernel_9552\955576227.py:22: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  TI_timeseries.at[i,"res_elec_app"] = STEM_res_ELEC_APP_WD.

In [8]:
path = str(DATA_DIRECTORY)+ "/results/TI_demand.csv"
filepath = pathlib.Path(path)  
filepath.parent.mkdir(parents=True, exist_ok=True)  
TI_timeseries.to_csv(filepath, sep=";", encoding='utf-8-sig', index_label='index')

In [9]:
sat = TI_timeseries['dayofweek'].value_counts().get("6", 0)/24
sun = TI_timeseries['dayofweek'].value_counts().get("0", 0)/24
days = len(TI_timeseries)/24 
print(sat, sun, days)

52.0 52.0 366.0


In [10]:
TI_timeseries

,datetime,formatted_date,dayofweek,month,hour,res_sh,res_dhw,res_elec_app,res_elec_light,ind_sh,ind_dhw,ind_hth,ind_elec,ser_sh_dhw,ser_elec,agr_elec
0,2020-01-01 00:00:00,01/01/2020 00:00,3,Jan,0,0.002213,0.000103,0.001067,0.002637,0.003417,0.003417,0.002973,NaN,0.003417,NaN,NaN
1,2020-01-01 01:00:00,01/01/2020 01:00,3,Jan,1,0.002330,0.000010,0.000767,0.001758,0.000000,0.003530,0.003017,NaN,0.003530,NaN,NaN
2,2020-01-01 02:00:00,01/01/2020 02:00,3,Jan,2,0.002537,0.000147,0.000633,0.001318,0.003557,0.003557,0.003003,NaN,0.003557,NaN,NaN
3,2020-01-01 03:00:00,01/01/2020 03:00,3,Jan,3,0.003290,0.000093,0.000633,0.001099,0.003800,0.003800,0.003017,NaN,0.003800,NaN,NaN
4,2020-01-01 04:00:00,01/01/2020 04:00,3,Jan,4,0.004913,0.000103,0.000733,0.001099,0.004180,0.004180,0.003223,NaN,0.004180,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8779,2020-12-31 19:00:00,31/12/2020 19:00,4,Dec,19,0.005070,0.006473,0.002867,0.010327,0.003807,0.003807,0.002900,NaN,0.003807,NaN,NaN
8780,2020-12-31 20:00:00,31/12/2020 20:00,4,Dec,20,0.004400,0.003470,0.002433,0.009888,0.003557,0.003557,0.002943,NaN,0.003557,NaN,NaN
8781,2020-12-31 21:00:00,31/12/2020 21:00,4,Dec,21,0.002947,0.002453,0.001833,0.008789,0.003290,0.003290,0.002910,NaN,0.003290,NaN,NaN
8782,2020-12-31 22:00:00,31/12/2020 22:00,4,Dec,22,0.002300,0.003157,0.001533,0.007251,0.003110,0.003110,0.002893,NaN,0.003110,NaN,NaN


In [11]:
TI_path = str(DATA_DIRECTORY)+ "/results/TI_demand.xlsx"

In [12]:
TI_demand_timeseries = pd.read_excel(TI_path, header=0, sheet_name ="TI_demand_timeseries")
TI_demand_timeseries

,index,datetime,formatted_date,dayofweek,month,hour,res_sh,res_dhw,res_elec_app,res_elec_light,...,ser_elec,agr_elec,res_mob_gasoline,res_mob_diesel,res_mob_elec,res_mob_gas,ser_mob_gasoline,ser_mob_diesel,agr_mob_diesel,tram
0,0,2020-01-01 00:00:00,2020-01-01 00:00:00,3,Jan,0,0.002213,0.000103,0.001067,0.002637,...,0.003417,0.003417,0.000019,0.000019,0.000121,0.000019,0.000114,0.000114,0.000114,0.000000
1,1,2020-01-01 01:00:00,2020-01-01 01:00:00,3,Jan,1,0.002330,0.000010,0.000767,0.001758,...,0.003530,0.003530,0.000010,0.000010,0.000075,0.000010,0.000114,0.000114,0.000114,0.000000
2,2,2020-01-01 02:00:00,2020-01-01 02:00:00,3,Jan,2,0.002537,0.000147,0.000633,0.001318,...,0.003557,0.003557,0.000000,0.000000,0.000046,0.000000,0.000114,0.000114,0.000114,0.000000
3,3,2020-01-01 03:00:00,2020-01-01 03:00:00,3,Jan,3,0.003290,0.000093,0.000633,0.001099,...,0.003800,0.003800,0.000000,0.000000,0.000024,0.000000,0.000114,0.000114,0.000114,0.000000
4,4,2020-01-01 04:00:00,2020-01-01 04:00:00,3,Jan,4,0.004913,0.000103,0.000733,0.001099,...,0.004180,0.004180,0.000010,0.000010,0.000013,0.000010,0.000114,0.000114,0.000114,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8779,8779,2020-12-31 19:00:00,2020-12-31 19:00:00,4,Dec,19,0.005070,0.006473,0.002867,0.010327,...,0.003807,0.003807,0.000173,0.000173,0.000204,0.000173,0.000114,0.000114,0.000114,0.000141
8780,8780,2020-12-31 20:00:00,2020-12-31 20:00:00,4,Dec,20,0.004400,0.003470,0.002433,0.009888,...,0.003557,0.003557,0.000096,0.000096,0.000199,0.000096,0.000114,0.000114,0.000114,0.000141
8781,8781,2020-12-31 21:00:00,2020-12-31 21:00:00,4,Dec,21,0.002947,0.002453,0.001833,0.008789,...,0.003290,0.003290,0.000077,0.000077,0.000183,0.000077,0.000114,0.000114,0.000114,0.000141
8782,8782,2020-12-31 22:00:00,2020-12-31 22:00:00,4,Dec,22,0.002300,0.003157,0.001533,0.007251,...,0.003110,0.003110,0.000038,0.000038,0.000164,0.000038,0.000114,0.000114,0.000114,0.000141


In [29]:
TI = pd.read_excel(TI_path, header=0, index_col=0, sheet_name ="TI_demand_annual")
TI_demand_annual = TI.iloc[3:]
TI_demand_annual

,Bellinzona,Blenio,Leventina,Locarno,Lugano,Mendrisio,Riviera,Vallemaggia,TI
index,,,,,,,,,
res_elec_app,2.210182e+07,2.840994e+06,4.425652e+06,2.986902e+07,6.359880e+07,2.023200e+07,4.118517e+06,2.840605e+06,1.500274e+08
res_elec_light,7.735638e+07,9.943478e+06,1.548978e+07,1.045416e+08,2.225958e+08,7.081198e+07,1.441481e+07,9.942117e+06,5.250959e+08
res_sh,3.881450e+08,5.755207e+07,9.700613e+07,3.163994e+08,8.142494e+08,2.787720e+08,8.986406e+07,4.185786e+07,2.083846e+09
res_dhw,6.982699e+07,7.166092e+06,1.262213e+07,9.322201e+07,2.186806e+08,7.339828e+07,1.269510e+07,7.041705e+06,4.946529e+08
ind_sh,5.431119e+07,1.237361e+06,3.609950e+07,2.780999e+07,6.019938e+07,4.297798e+07,1.322245e+07,2.500217e+06,2.383581e+08
ind_dhw,4.626261e+05,8.125200e+03,2.366739e+05,3.720699e+05,7.283565e+05,5.835366e+05,8.593830e+04,2.272950e+04,2.500056e+06
ind_hth,3.919093e+07,6.754859e+06,9.558763e+06,6.429861e+07,1.445285e+08,6.493586e+07,1.064209e+07,1.013229e+07,3.500419e+08
ser_sh_dhw,1.088805e+08,1.721944e+07,2.855267e+07,1.132685e+08,3.146288e+08,8.069997e+07,1.615927e+07,1.284081e+07,6.922499e+08
agr_elec,1.072536e+07,7.643360e+06,7.088600e+06,1.208144e+07,1.571820e+07,1.146504e+07,2.835440e+06,4.438080e+06,7.199552e+07


In [ ]:
def make_csv(df):
    # iterating the columns
    
        print(col)
        re_sh[col] =  TI_demand_annual.loc["res_sh", col]*TI_demand_timeseries["res_sh"]
    re_sh

In [32]:
for ind in TI_demand_annual.index:
    print(ind)

res_elec_app
res_elec_light
res_sh
res_dhw
ind_sh
ind_dhw
ind_hth
ser_sh_dhw
agr_elec
ind_elec
ser_elec
res_mob_gasoline
res_mob_diesel
res_mob_elec
res_mob_gas
ser_mob_gasoline
ser_mob_diesel
agr_mob_diesel


In [33]:
df = pd.DataFrame()

# iterating the columns
for d_type in TI_demand_annual.index:
    #print(d_type)
    for col in TI_demand_annual.columns:
        #print(col)
        df[col] =  TI_demand_annual.loc[d_type, col]*TI_demand_timeseries[d_type]

    path = str(DATA_DIRECTORY)+ "/results/" + d_type + ".csv"
    filepath = pathlib.Path(path)  
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    df.to_csv(filepath, sep=";", encoding='utf-8-sig', index_label='index')
    df = df[0:0]

In [15]:
TI_demand_annual.loc["res_sh"]

Bellinzona     3.881450e+08
Blenio         5.755207e+07
Leventina      9.700613e+07
Locarno        3.163994e+08
Lugano         8.142494e+08
Mendrisio      2.787720e+08
Riviera        8.986406e+07
Vallemaggia    4.185786e+07
TI             2.083846e+09
Name: res_sh, dtype: float64

In [21]:
TI_demand_annual.loc["res_sh", "Bellinzona"]

388144992.9

Bellinzona
Blenio
Leventina
Locarno
Lugano
Mendrisio
Riviera
Vallemaggia
TI


In [16]:
TI_demand_timeseries["res_sh"]

0       0.002213
1       0.002330
2       0.002537
3       0.003290
4       0.004913
          ...   
8779    0.005070
8780    0.004400
8781    0.002947
8782    0.002300
8783    0.002167
Name: res_sh, Length: 8784, dtype: float64

In [34]:
TI_demand_timeseries["res_mob_gas"]

0       0.000019
1       0.000010
2       0.000000
3       0.000000
4       0.000010
          ...   
8779    0.000173
8780    0.000096
8781    0.000077
8782    0.000038
8783    0.000019
Name: res_mob_gas, Length: 8784, dtype: float64

In [20]:
test = pd.DataFrame()
test["B"] =  TI_demand_annual.loc["res_sh", "Bellinzona"]*TI_demand_timeseries["res_sh"]
test

,B
0,8.590941e+05
1,9.043778e+05
2,9.845946e+05
3,1.276997e+06
4,1.907086e+06
...,...
8779,1.967895e+06
8780,1.707838e+06
8781,1.143734e+06
8782,8.927335e+05
